In [2]:
from pathlib import Path
import pandas as pd
from pandas import DataFrame, MultiIndex, merge, read_csv, concat


from utils.data import get_dataframe, remove_warmup_df
from utils.global_config import (
    N_WARMUPS,
    SERIES,
    TSTEPS,
    DATA_DIR,
    TEST_FILE,
    METRIC_CSV_FILE,
)
from utils.post_processing import METRIC, DSET, TGT, VAL

# from utils.errors_computing import compute_errors_df

# from utils.errors_computing_train import compute_errors_df_omg_so_ugly

In [41]:
dft = read_csv("AF_FE/" + "/" + METRIC_CSV_FILE, index_col=0)
dft

,dataset,target,metric,value
0,train,y_fixed,mean_absolute_error,0.138
1,train,y_fixed,mean_squared_error,0.0350
2,train,y_fixed_obs,mean_absolute_error,0.796
3,train,y_fixed_obs,mean_squared_error,0.995
4,test,y_fixed,mean_absolute_error,0.187
5,test,y_fixed,mean_squared_error,0.0744
6,test,y_fixed_obs,mean_absolute_error,0.820
7,test,y_fixed_obs,mean_squared_error,1.06
0,test,y_fixed,quad-bias,0.0316
1,test,y_fixed_obs,quad-bias,1.01


In [42]:
print(dft.style.to_html())

<style type="text/css">
</style>
<table id="T_fcf90">
  <thead>
    <tr>
      <th class="blank level0" >&nbsp;</th>
      <th id="T_fcf90_level0_col0" class="col_heading level0 col0" >dataset</th>
      <th id="T_fcf90_level0_col1" class="col_heading level0 col1" >target</th>
      <th id="T_fcf90_level0_col2" class="col_heading level0 col2" >metric</th>
      <th id="T_fcf90_level0_col3" class="col_heading level0 col3" >value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th id="T_fcf90_level0_row0" class="row_heading level0 row0" >0</th>
      <td id="T_fcf90_row0_col0" class="data row0 col0" >train</td>
      <td id="T_fcf90_row0_col1" class="data row0 col1" >y_fixed</td>
      <td id="T_fcf90_row0_col2" class="data row0 col2" >mean_absolute_error</td>
      <td id="T_fcf90_row0_col3" class="data row0 col3" >0.138479</td>
    </tr>
    <tr>
      <th id="T_fcf90_level0_row1" class="row_heading level0 row1" >1</th>
      <td id="T_fcf90_row1_col0" class="data row1 col0" >train<

In [58]:
def apply_style(df):
    return dft.style.format("{:#.3g}", subset=["value"]).set_table_styles(
        [
            {
                "selector": "table, th, td",
                "props": "border: 1px solid; text-align: center",
            },
        ],
        overwrite=True,
    )


apply_style(dft)

,dataset,target,metric,value
0,train,y_fixed,mean_absolute_error,0.138
1,train,y_fixed,mean_squared_error,0.0350
2,train,y_fixed_obs,mean_absolute_error,0.796
3,train,y_fixed_obs,mean_squared_error,0.995
4,test,y_fixed,mean_absolute_error,0.187
5,test,y_fixed,mean_squared_error,0.0744
6,test,y_fixed_obs,mean_absolute_error,0.820
7,test,y_fixed_obs,mean_squared_error,1.06
0,test,y_fixed,quad-bias,0.0316
1,test,y_fixed_obs,quad-bias,1.01


In [77]:
STDY = "study"


def apply_style(df: pd.DataFrame):
    stlr = df.style
    stlr.format("{:#.3g}")
    stlr.set_table_styles(
        [
            {
                "selector": "table, th, td",
                "props": "border: 1px solid; text-align: center",
            },
        ]
    )
    # stlr.background_gradient(axis=0)
    return stlr


# def summarize_df(list_dir: list[str]) -> DataFrame:

#     df = DataFrame()
#     for resdir in list_dir:
#         try:
#             dft = read_csv(resdir + "/" + METRIC_CSV_FILE, index_col=0)
#         except FileNotFoundError:
#             dft = read_csv(
#                 "../mixed_model_forecast/" + resdir + "/" + METRIC_CSV_FILE, index_col=0
#             )
#         dft[STDY] = resdir
#         df = concat([df, dft])

#     df = df.pivot(index=[METRIC, STDY], columns=[DSET, TGT], values=[VAL])
#     idx = df.index
#     df = df.reindex(list_dir, level=1)
#     display(df)
#     print("\n\n======\n\n")
#     return df


def summarize_df(list_dir: list[str]) -> DataFrame:

    df = DataFrame()
    for resdir in list_dir:
        try:
            dft = read_csv(resdir + "/" + METRIC_CSV_FILE, index_col=0)
        except FileNotFoundError:
            dft = read_csv(
                "../mixed_model_forecast/" + resdir + "/" + METRIC_CSV_FILE, index_col=0
            )
        dft = dft[dft[METRIC] != "mean_absolute_error"]
        dft[METRIC] = dft[METRIC].str.replace("mean_squared_error", "MSE")
        dft[STDY] = resdir
        df = concat([df, dft])

    df = df.pivot(index=[STDY], columns=[DSET, METRIC, TGT], values=[VAL])
    idx = df.index
    df = df.reindex(list_dir, axis=0)
    display(apply_style(df))


hlp = summarize_df(["SF_FE", "AF_FE"])

In [78]:
summarize_df(
    ["oracle", "linear", "time-polynom", "SF_ME", "AF_ME", "SF_ME_y", "AF_ME_y"]
)

# Reformater

In [7]:
pd.options.display.float_format = "{:#5.3g}".format
df = pd.read_csv("/home/francois/Téléchargements/table.csv", header=[0, 1])
df

,Unnamed: 0_level_0,train set,Unnamed: 2_level_0,test set,Unnamed: 4_level_0
,Model,without noise,with noise,without noise,with noise
0,Oracle (marg),280.,281.,246.,247.
1,Linear mixed (marg),376.,377.,315.,316.
2,t-polynomial (marg),464.,465.,373.,373.
3,ODE RNN,310.,263.,296.,297.
4,RNN,298.,254.,289.,290.
5,Mixed Reservoir* (marg.),NaN,NaN,170.,172.
6,Oracle (ss),0.0850,0.915,0.0840,0.908
7,Linear mixed (subject specific),0.162,0.894,0.162,0.884
8,t-polynomial (subject specific),8.22,9.07,8.29,9.05


In [10]:
df.to_latex()

'\\begin{tabular}{llrrrr}\n\\toprule\n & Unnamed: 0_level_0 & train set & Unnamed: 2_level_0 & test set & Unnamed: 4_level_0 \\\\\n & Model & without noise & with noise & without noise & with noise \\\\\n\\midrule\n0 & Oracle (marg) & 279.856000 & 280.803000 & 245.712000 & 246.598000 \\\\\n1 & Linear mixed (marg) & 375.664000 & 376.581000 & 315.035000 & 316.067000 \\\\\n2 & t-polynomial (marg) & 463.831000 & 464.751000 & 372.615000 & 373.340000 \\\\\n3 & ODE RNN & 309.716000 & 262.709000 & 296.221000 & 297.136000 \\\\\n4 & RNN & 297.875000 & 254.463000 & 289.292000 & 290.100000 \\\\\n5 & Mixed Reservoir* (marg.) & NaN & NaN & 169.700000 & 172.200000 \\\\\n6 & Oracle (ss) & 0.085000 & 0.915000 & 0.084000 & 0.908000 \\\\\n7 & Linear mixed (subject specific) & 0.162000 & 0.894000 & 0.162000 & 0.884000 \\\\\n8 & t-polynomial (subject specific) & 8.225000 & 9.067000 & 8.285000 & 9.055000 \\\\\n9 & VAE & 0.350000 & NaN & 1.670000 & NaN \\\\\n10 & VAE (noisy covariates) & 0.380000 & NaN & 1.8